<a href="https://colab.research.google.com/github/ameliachoi/tutorial-python-machine-learning/blob/master/python_ml_08_20newsgroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Tutorial] 20 News Groups 20 뉴스그룹 분류

---
### 텍스트 정규화

In [1]:
from sklearn.datasets import fetch_20newsgroups

data = fetch_20newsgroups(subset='all',
                          random_state = 156)

In [2]:
print(data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


* fetch_20newsgroups는 다른 데이터 세트 예제와 같이 파이썬 딕셔너리와 유사한 Bunch 객체를 반환합니다.
* key 값 중에서, `filenames`는 로컬 컴퓨터에 저장하는 디렉터리와 파일명을 지칭합니다.

In [3]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


* `target` 클래스의 값은 0부터 19까지 20개로 구성되어 있습니다.
* 그리고 하단의 클래스 이름과 짝지어 연결되어 있습니다.

In [4]:
# 데이터 하나만 추출해 값 확인하기
print(data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

* 텍스트 데이터를 확인해보면, 뉴스그룹 기사 내용뿐만 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등 다양한 정보를 포함하고 있습니다.
* 이 중에서 내용을 제외하고 제목 등의 다른 정보는 제거합니다.
* 제목과 소속, 이메일 주소와 같은 정보는 target 클래스 값과 유사한 데이터를 가지고 있는 경우가 상당히 많기 때문에, 왠만한 머신러닝 알고리즘을 적용해도 상당히 높은 예측 성능을 나타냅니다.
* remove 파라미터를 이용해 header, footer를 제거할 수 있습니다.

In [5]:
from sklearn.datasets import fetch_20newsgroups

# subset = 'train'으로 학습용 데이터만 추출, remove 사용
train = fetch_20newsgroups(subset = 'train',
                           remove = ('headers', 'footers', 'quotes'),
                           random_state=156)
x_train = train.data
y_train = train.target

test = fetch_20newsgroups(subset = 'test',
                           remove = ('headers', 'footers', 'quotes'),
                           random_state=156)
x_test = test.data
y_test = test.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train.data),
                                             len(test.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


---

### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

* CountVectorizer를 이용하여 학습/테스트 데이터의 텍스트를 피처 벡터화
* 유의 사항은, 테스트 데이터에서 CountVectorizer를 이용할 때, 반드시 **학습 데이터를 이용해** fit()이 수행된 CountVectorizer 객체를 이용해 데이터를 transform 해야 한다는 것입니다.
* 그래야만 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 CountVectorize로 변환할 피처 개수가 같아집니다.


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# 피처 벡터화 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(x_train)
x_train_cnt_vect = cnt_vect.transform(x_train)

# 학습 데이터로 fit된 countvectorizer를 이용하여 테스트 데이터 피처 벡터화 변환 수행
x_test_cnt_vect = cnt_vect.transform(x_test)

print('학습 데이터 텍스트의 CountVectorizer Shape: ', x_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape:  (11314, 101631)


In [8]:
# 로지스틱 회귀 이용해 뉴스그룹에 대한 분류 예측

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# logistic regression 수행
lr = LogisticRegression()
lr.fit(x_train_cnt_vect, y_train)
pred = lr.predict(x_test_cnt_vect)
print('CountVectorizer Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorizer Logistic Regression의 예측 정확도는 0.608


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [10]:
# TF-IDF 기반으로 벡터화 변경
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(x_train)
x_train_tfidf_vect = tfidf_vect.transform(x_train)
x_test_tfidf_vect = tfidf_vect.transform(x_test)

# logistic regression 이용해 학습/예측/평가 수행
lr = LogisticRegression()
lr.fit(x_train_tfidf_vect, y_train)
pred = lr.predict(x_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


* TF-IDF 방식이 단순 카운트 기반보다 훨씬 높은 예측 정확도를 제공합니다.
* 다양한 파라미터를 적용하여 정확도를 높여보겠습니다.

In [11]:
# stop words 필터링을 추가하고 ngram을 기본 (1, 1)에서 (1, 2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english',
                             ngram_range=(1,2),
                             max_df = 300)
tfidf_vect.fit(x_train)
x_train_tfidf_vect = tfidf_vect.transform(x_train)
x_test_tfidf_vect = tfidf_vect.transform(x_test)

lr = LogisticRegression()
lr.fit(x_train_tfidf_vect, y_train)
pred = lr.predict(x_test_tfidf_vect)
print('IF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

IF-IDF Vectorized Logistic Regression의 예측 정확도는 0.692


In [12]:
# GridSearchCV를 이용하여 로지스틱 회귀 하이퍼 파라미터 최적화 작업 진행
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 수행
params = {'C' : [0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr, param_grid = params,
                          cv = 3, scoring = 'accuracy', verbose = 1)
grid_cv_lr.fit(x_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter :', grid_cv_lr.best_params_)

# 최적 C값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(x_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
 

Logistic Regression best C parameter : {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.701


---
### 사이킷런 파이프라인 사용 및 GridSearchCV와 결합

* sklearn의 Pipeline 클래스를 이용하면 피처 벡터화와 알고리즘 학습/예측을 위한 코드 작성을 한 번에 진행할 수 있습니다.
* 데이터 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리할 수 있어 더 직관적인 ML 모델 코드를 생성할 수 있습니다.
* 또한, 대용량 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 스트림 기반에서 바로 알고리즘의 데이터로 입력할 수 있기 때문에 수행 시간을 절약할 수 있습니다.

In [13]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english',
                                                    ngram_range=(1,2),
                                                    max_df=300)),
                     ('lr', LogisticRegression(C=10))])

# 별도의 TfidfVectorizer의 fit, transform과 LogisticRegression의 fit, transform이 필요없음
pipeline.fit(x_train, y_train)
pred = pipeline.predict(x_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


In [ ]:
pipeline = Pipeline([
                     ('tfidf_vect', TfidfVectorizer(stop_words='english')),
                     ('lr', LogisticRegression())
])

# pipeline에 기술된 각각의 객체 변수에 언더바 2개를 연달아 gridsearchCV에 사용될 파라미터/하이퍼 파라미터 이름과 값을 설정
params = {
    'tfidf_vect__ngram_range' : [(1,1), (1,2), (1,3)],
    'tfidf_vect__max_df' : [100, 300, 700],
    'lr__C' : [1, 5, 10]
}

# GridSearchCV의 생성자에 estimator가 아닌 pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline,
                            param_grid = params,
                            cv=3,
                            scoring='accuracy',
                            verbose=1)
grid_cv_pipe.fit(x_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(x_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
 